In [108]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import plotly.express as px
import os
from tensorflow.keras.metrics import Recall,AUC
from tensorflow.keras.utils import plot_model
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout

In [109]:
# print(len(os.listdir('/kaggle/input/not-fire-15k/Not_Fire')))

In [110]:
df = pd.DataFrame(columns=['path', 'label'])
path_fire = '/kaggle/input/fire-pp3'
path_not_fire = '/kaggle/input/not-fire-15k/Not_Fire'

fire_data = []
not_fire_data = []

for dirname, _, filenames in os.walk(path_fire):
    for filename in filenames:
        fire_data.append([os.path.join(dirname, filename), 'fire'])

for dirname, _, filenames in os.walk(path_not_fire):
    for filename in filenames:
        not_fire_data.append([os.path.join(dirname, filename), 'non_fire'])

fire_df = pd.DataFrame(fire_data, columns=['path', 'label'])
not_fire_df = pd.DataFrame(not_fire_data, columns=['path', 'label'])

df = pd.concat([fire_df, not_fire_df]).sample(frac=1).reset_index(drop=True)
df = df.sample(frac=1).reset_index(drop=True)
df.head(10)

,path,label
0,/kaggle/input/fire-pp3/fire.33045.jpg,fire
1,/kaggle/input/fire-pp3/fire.63954.jpg,fire
2,/kaggle/input/fire-pp3/fire.16726.jpg,fire
3,/kaggle/input/not-fire-15k/Not_Fire/non-fire.1...,non_fire
4,/kaggle/input/fire-pp3/fire.1763.jpg,fire
5,/kaggle/input/fire-pp3/fire.33160.jpg,fire
6,/kaggle/input/fire-pp3/fire.46084.jpg,fire
7,/kaggle/input/fire-pp3/fire.34642.jpg,fire
8,/kaggle/input/fire-pp3/fire.83135.jpg,fire
9,/kaggle/input/fire-pp3/fire.68061.jpg,fire


In [111]:
df1 = pd.DataFrame(columns=['path', 'label'])
path_fire1 = '/kaggle/input/fire-2k/fire'
path_not_fire1 = '/kaggle/input/no-fire-3-5k/no_fire_3'

fire_data1 = []
not_fire_data1 = []

for dirname, _, filenames in os.walk(path_fire1):
    for filename in filenames:
        fire_data1.append([os.path.join(dirname, filename), 'fire'])

for dirname, _, filenames in os.walk(path_not_fire1):
    for filename in filenames:
        not_fire_data1.append([os.path.join(dirname, filename), 'non_fire'])

fire_df1 = pd.DataFrame(fire_data1, columns=['path', 'label'])
not_fire_df1 = pd.DataFrame(not_fire_data1, columns=['path', 'label'])

df1 = pd.concat([fire_df1, not_fire_df1]).sample(frac=1).reset_index(drop=True)
df1 = df1.sample(frac=1).reset_index(drop=True)
df1.head(10)

,path,label
0,/kaggle/input/no-fire-3-5k/no_fire_3/non-fire....,non_fire
1,/kaggle/input/fire-2k/fire/file.703.jpg,fire
2,/kaggle/input/no-fire-3-5k/no_fire_3/non-fire....,non_fire
3,/kaggle/input/no-fire-3-5k/no_fire_3/non-fire....,non_fire
4,/kaggle/input/no-fire-3-5k/no_fire_3/non-fire....,non_fire
5,/kaggle/input/no-fire-3-5k/no_fire_3/non-fire....,non_fire
6,/kaggle/input/fire-2k/fire/file.1029.jpg,fire
7,/kaggle/input/no-fire-3-5k/no_fire_3/non-fire....,non_fire
8,/kaggle/input/no-fire-3-5k/no_fire_3/non-fire....,non_fire
9,/kaggle/input/no-fire-3-5k/no_fire_3/non-fire....,non_fire


In [112]:
generator = ImageDataGenerator(
    rotation_range= 20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range = 2,
    zoom_range=0.2,
    rescale = 1/255,
    validation_split=0.2,
)

In [113]:
train_gen = generator.flow_from_dataframe(df,x_col='path',y_col='label',images_size=(256,256),class_mode='binary',subset='training', batch_size=32)
val_gen = generator.flow_from_dataframe(df1,x_col='path',y_col='label',images_size=(256,256),class_mode='binary',subset='validation', batch_size=32)

Found 77656 validated image filenames belonging to 2 classes.
Found 1347 validated image filenames belonging to 2 classes.


In [114]:
class_indices = {}
for key in train_gen.class_indices.keys():
    class_indices[train_gen.class_indices[key]] = key

print(class_indices)

{0: 'fire', 1: 'non_fire'}


### sns.set_style('dark')
pics = 6 #set the number of pics
fig,ax = plt.subplots(int(pics//2),2,figsize=(15,15))
plt.suptitle('Generated images in training set')
ax = ax.ravel()
for i in range((pics//2)*2):
    ax[i].imshow(train_gen[0][0][i])
    ax[i].axes.xaxis.set_visible(False)
    ax[i].axes.yaxis.set_visible(False)

In [121]:
model = Sequential()
model.add(Conv2D(filters=16,kernel_size = (3,3),activation='relu',input_shape = (256,256,3)))
model.add(MaxPool2D())
for i in range(4):
  num = 2**(i + 8)
  model.add(Conv2D(filters=num,kernel_size=(3,3),padding='same',activation='relu'))
  model.add(MaxPool2D())
# model.add(Conv2D(filters=32,kernel_size=(3,3),activation='relu'))
# model.add(MaxPool2D())
# model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu'))
# model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(64,activation='relu')) 
model.add(Dense(32,activation = 'relu'))
model.add(Dense(1,activation = 'sigmoid'))


# model = Sequential()
# model.add(Conv2D(filters=16,kernel_size = (3,3),activation='relu', padding='same',input_shape = (256,256,3)))
# model.add(MaxPool2D())
# for i in range(4):
#   num = 2**(i + 5)
#   model.add(Conv2D(filters=num,kernel_size=(3,3),padding='same',activation='relu'))
#   model.add(MaxPool2D())
# model.add(Flatten())
# model.add(Dense(64,activation='relu'))
# model.add(Dense(32,activation = 'relu'))
# model.add(Dense(2,activation = 'softmax'))


In [122]:
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


In [123]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_41 (Conv2D)          (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d_37 (MaxPoolin  (None, 127, 127, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_42 (Conv2D)          (None, 127, 127, 256)     37120     
                                                                 
 max_pooling2d_38 (MaxPoolin  (None, 63, 63, 256)      0         
 g2D)                                                            
                                                                 
 conv2d_43 (Conv2D)          (None, 63, 63, 512)       1180160   
                                                                 
 max_pooling2d_39 (MaxPoolin  (None, 31, 31, 512)     

In [124]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy',Recall(),AUC()])

In [125]:
early_stoppping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr_on_plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.1,patience=5)

In [ ]:
model.fit(x=train_gen, steps_per_epoch=train_gen.samples//32,epochs=2, validation_data=val_gen, validation_steps=val_gen.samples//32, callbacks=[early_stoppping,reduce_lr_on_plateau])

Epoch 1/2
2426/2426 [==============================] - 1359s 554ms/step - loss: 0.1883 - accuracy: 0.9275 - recall_8: 0.5230 - auc_8: 0.9362 - val_loss: 0.2783 - val_accuracy: 0.8958 - val_recall_8: 0.8642 - val_auc_8: 0.9740 - lr: 0.0010
Epoch 2/2
 677/2426 [=======>......................] - ETA: 15:16 - loss: 0.1088 - accuracy: 0.9622 - recall_8: 0.7936 - auc_8: 0.9786

In [ ]:
history= model.history.history
px.line(history,title = "Metrics Plot")
# fig.show()
# print(history)

In [ ]:

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history['loss'], label='Train Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

plt.subplot(1, 2, 2)
plt.plot(history['accuracy'], label='Train Accuracy')
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')

plt.show()


In [ ]:
evaluation = model.evaluate(val_gen)
print("Loss: {:.5f}".format(evaluation[0]))
print("Accuracy: {:.2f}%".format(evaluation[1] * 100))


In [ ]:
xception = Xception(include_top = False,input_shape = (256,256,3))
input_to_model = xception.input
#turn off training
xception.trainable = False

x = Flatten()(xception.output)
x = Dense(64,activation = 'relu')(x)
output_to_model = Dense(1,activation = 'sigmoid')(x)
model2 = Model(inputs = input_to_model,outputs = output_to_model)

In [ ]:
model2.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy',Recall(),AUC()])

In [ ]:
history2 = model2.fit(x=train_gen, steps_per_epoch=train_gen.samples//32,epochs=2, validation_data=val_gen, validation_steps=val_gen.samples//32, callbacks=[early_stoppping,reduce_lr_on_plateau])

In [ ]:
px.line(history,title='Metrics Plot')

In [ ]:
eval_list = model2.evaluate(val_gen,return_dict=True)
for metric in eval_list.keys():
    print(metric+f": {eval_list[metric]:.2f}")

In [ ]:
model2.save('fire_detection_model_ver_demo_special_P3_kaggle_Lam_v13_6.h5')